# TensorFlow 입문

## TensorFlow 설치와 동작 확인하기

In [1]:
import tensorflow as tf
sess = tf.Session()
hello = tf.constant('Hello')
print(sess.run(hello))

b'Hello'


## TensorFlow 데이터 플로 그래프

### 덧셈하는 데이터 플로 그래프

In [2]:
# TensorFlow 읽어들이기
import tensorflow as tf

# 상수 정의하기
a = tf.constant(100)
b = tf.constant(30)

# 연산 정의하기
add_op = a + b

# 세션 시작하기
sess = tf.Session()
res = sess.run(add_op)  # 식 평가하기
print(res)

130


### 곱셈하는 데이터 플로 그래프

In [3]:
# TensorFlow 읽어들이기
import tensorflow as tf

# 상수 정의하기
a = tf.constant(10)
b = tf.constant(20)
c = tf.constant(30)

# 연산 정의하기
mul_op = (a + b) * c

# 세션 시작하기
sess = tf.Session()
res = sess.run(mul_op)  # 식 평가하기
print(res)

900


### 데이터 플로 그래프를 그림으로 확인하기

In [4]:
# TensorFlow 읽어들이기
import tensorflow as tf

# 상수 정의하기
a = tf.constant(10, name='10')
b = tf.constant(20, name='20')
c = tf.constant(30, name='30')

# 연산 정의하기
add_op = tf.add(a,b, name='add')
mul_op = tf.multiply(add_op, c, name='mul')

# 세션 시작하기
sess = tf.Session()
res = sess.run(mul_op)  # 식 평가하기
print(res)

# 텐서플로로 그래프 출력하기
tf.summary.FileWriter('.logs', sess.graph)

900


### 변수 사용하기

In [5]:
import tensorflow as tf

# 변수 정의하기
v = tf.Variable(0, name='v')

# 상수 정의하기
a = tf.constant(10, name='10')
b = tf.constant(20, name='20')

# 연산 정의하기
mul_op = tf.multiply(a, b, name='mul')
assign_op = tf.assign(v, mul_op)

# 세션 시작하기
sess = tf.Session()
# 연산 실행하기
sess.run(assign_op)

# 텐서플로로 그래프 출력하기
tf.summary.FileWriter('.logs', sess.graph)

#결과 출력하기
res = sess.run(v)
print(res)

200


### 플레이스홀더 사용하기

In [6]:
import tensorflow as tf

# 플레이스홀더 정의하기 
a = tf.placeholder(tf.int32, [5])

# 벡터를 2배 하는 연산 정의하기
two = tf.constant(2)
x2_op = a * two

# 세션 시작하기 
sess = tf.Session()

# 플레이스 홀더에 값을 넣어 실행하기 
res1 = sess.run(x2_op, feed_dict={ a: [1, 2, 3, 4, 5] })
print(res1)
res2 = sess.run(x2_op, feed_dict={ a: [5, 6, 7, 10, 100] })
print(res2)

[ 2  4  6  8 10]
[ 10  12  14  20 200]


In [7]:
import tensorflow as tf

# 플레이스홀더 정의하기 
a = tf.placeholder(tf.int32, [None, 2])

# 벡터를 2배 하는 연산 정의하기 
two = tf.constant(2)
x2_op = a * two

# 세션 시작하기 
sess = tf.Session()

# 플레이스 홀더에 값을 넣어 실행하기 
sample_list = [[1, 1], [2, 2], [3, 3], [4, 4]]
res = sess.run(x2_op, feed_dict={ a: sample_list })
print(res)

[[2 2]
 [4 4]
 [6 6]
 [8 8]]


# TensorFlow로 붓꽃 분류하기

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

# 붓꽃 데이터 읽어 들이기
iris_data = pd.read_csv("iris.csv", encoding="utf-8")

# 붓꽃 데이터를 레이블과 입력 데이터로 분리하기
y_labels = iris_data.loc[:,"Name"]
x_data = iris_data.loc[:,["SepalLength","SepalWidth","PetalLength","PetalWidth"]]

## 레이블을 One-hot 벡터 형식으로 바꾸기

In [3]:
# 레이블 데이터를 One-hot 형식으로 변환하기
labels = {
    'Iris-setosa': [1, 0, 0], 
    'Iris-versicolor': [0, 1, 0], 
    'Iris-virginica': [0, 0, 1]
}
y_nums = list(map(lambda v : labels[v] , y_labels))

## 테스트 데이터를 학습전용 데이터와 테스트 전용 데이터로 분활하기

In [5]:
# 학습 전용과 테스트 전용 분리하기
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_nums, train_size=0.8)

## 학습 알고리즘 정의하기

In [6]:
# 붓꽃 데이터의 입력값(4차원)과 출력값(3차원)을 넣을 위치 정의하기
x  = tf.placeholder(tf.float32, [None, 4])
y_ = tf.placeholder(tf.float32, [None, 3])

# 가중치와 바이어스 변수 정의하기
w = tf.Variable(tf.zeros([4, 3])) # 가중치
b = tf.Variable(tf.zeros([3])) # 바이어스

# 소프트맥스 회귀 정의하기
y = tf.nn.softmax(tf.matmul(x, w) + b)

# 모델 훈련하기
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.AdamOptimizer(0.05)
train = optimizer.minimize(cross_entropy)

# 정답률 구하기
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

## 테스트 학습 및 실행 테스트

In [8]:
# 세션 시작하고 변수 초기화하기
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 학습하기
train_feed_dict = {x: x_train, y_: y_train}
for step in range(300):
    sess.run(train, feed_dict=train_feed_dict)

# 테스트 테이터를 사용해서 예측하고 최종 정답률 구하기
acc = sess.run(accuracy, feed_dict={x: x_test, y_: y_test})
print("정답률=", acc)

정답률= 0.96666664


## keras

In [9]:
import tensorflow.contrib.keras as keras
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 붓꽃 데이터 읽어 들이기
iris_data = pd.read_csv("iris.csv", encoding="utf-8")

# 붓꽃 데이터를 레이블과 입력 데이터로 분리하기
y_labels = iris_data.loc[:,"Name"]
x_data = iris_data.loc[:,["SepalLength","SepalWidth","PetalLength","PetalWidth"]]

# 레이블 데이터를 One-hot 형식으로 변환하기
labels = {
    'Iris-setosa': [1, 0, 0], 
    'Iris-versicolor': [0, 1, 0], 
    'Iris-virginica': [0, 0, 1]
}
y_nums = np.array(list(map(lambda v : labels[v] , y_labels)))
x_data = np.array(x_data)

# 학습 전용과 테스트 전용 분리하기
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_nums, train_size=0.8)

# 모델의 구조 정의하기
Dense = keras.layers.Dense
model = keras.models.Sequential()
model.add(Dense(10, activation='relu', input_shape=(4,)))
model.add(Dense(3, activation='softmax'))

# 모델 구축하기
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# 학습 실행하기
model.fit(np.array(x_train), np.array(y_train),
    batch_size=20,
    epochs=300)
#    validation_data=(x_test, y_test))

# 데이터 평가하기
score = model.evaluate(x_test, y_test, verbose=1)
print('정답률=', score[1], 'loss=', score[0])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 120 samples
Epoch 1/300
120/120 [==============================] - 0s 989us/sample - loss: 3.3505 - acc: 0.3333
Epoch 2/300
120/120 [==============================] - 0s 108us/sample - loss: 3.0658 - acc: 0.3333
Epoch 3/300
120/120 [==============================] - 0s 150us/sample - loss: 2.7930 - acc: 0.3333
Epoch 4/300
120/120 [==============================] - 0s 92us/sample - loss: 2.5422 - acc: 0.3333
Epoch 5/300
120/120 [==============================] - 0s 108us/sample - loss: 2.3158 - acc: 0.3333
Epoch 6/300
120/120 [==============